# selenium-yahoo-finance-scraper

Use the "Run" button to execute the code.

In [1]:
!pip install jovian --upgrade --quiet
!pip install selenium --upgrade --quiet

##prereq

- !pip install webdriver-manager --upgrade --quiet
- download required chromdriver and place it in the project path 

![](https://imgur.com/caHD7Dm.png)

![](https://imgur.com/H2LfYlh.png)


![](https://imgur.com/7jMFOcE.png)


![](https://media.giphy.com/media/TwLrWWtF6zcDGk5BQf/giphy.gif)

In [2]:
import jovian

In [3]:
# Execute this to save new versions of the notebook
jovian.commit(project="selenium-yahoo-finance-scraper")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vinodvidhole/selenium-yahoo-finance-scraper" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper


'https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper'

In [4]:
!pip install webdriver-manager --upgrade --quiet

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [6]:
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')
    serv = Service(ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=serv)
    driver = webdriver.Chrome(options=chrome_options, service=serv)
    driver.get(YAHOO_FINANCE_URL)
    return driver

In [7]:
def get_table_rows(driver):
    TABLE_CLASS = "W(100%)"  
    tablerows = len(driver.find_elements(By.XPATH, value="//table[@class= '{}']/tbody/tr".format(TABLE_CLASS)))
    return tablerows

In [8]:
def get_table_header(driver):
    header = driver.find_elements(By.TAG_NAME, value= 'th')
    header_list = [item.text for index, item in enumerate(header) if index < 10]
    return header_list

In [9]:
'''
rownum = 1
colnum = 3
driver.find_element(By.XPATH, value="//tr[{}]/td[{}]".format(rownum,colnum)).text
'''

'\nrownum = 1\ncolnum = 3\ndriver.find_element(By.XPATH, value="//tr[{}]/td[{}]".format(rownum,colnum)).text\n'

In [10]:
def parse_table_rows(rownum, driver, header_list):
    row_dictionary = {}
    #"//tr[{}]/td[{}]"
    for index , item in enumerate(header_list):
        column_xpath = '//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{}]/td[{}]'.format(rownum, index+1)
        #row_dictionary[item] = driver.find_element(By.XPATH, value='//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{}]/td[{}]'.format(rownum, index+1)).text
        column_element = find_element_wait(driver, 1, (By.XPATH, column_xpath))
        row_dictionary[item] = column_element.text
    return row_dictionary

In [11]:
#(By.XPATH, next_button_xpath)
def find_element_wait(driver, wait_timer, locator):
    element = WebDriverWait(driver, wait_timer).until(EC.presence_of_element_located(locator))
    return element 

In [12]:
YAHOO_FINANCE_URL = 'https://finance.yahoo.com/cryptocurrencies'#'https://finance.yahoo.com/trending-tickers'

print('Creating driver')
driver = get_driver()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome


Creating driver


Driver [/Users/vinoddhole/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [13]:
header_list = get_table_header(driver)

In [14]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
TOTAL_CRYPTO = 100
table_data = []
next_button_xpath = '//*[@id="scr-res-table"]/div[2]/button[3]'
table_rows = get_table_rows(driver)
page_num = 1
is_scraping = True

while is_scraping:
    table_rows = get_table_rows(driver)
    print('Found {} rows on Page : {}'.format(table_rows, page_num))
    
    print('Parsing Page : {}'.format(page_num))
    table_data += [parse_table_rows(i, driver, header_list) for i in range (1, table_rows + 1)]
    total_count = len(table_data)
    print('Total rows scraped : {}'.format(total_count))
    if total_count > TOTAL_CRYPTO:
        print('Done Parsing..')
        is_scraping = False
    else:    
        print('Clicking Next Button')
        #element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
        element = find_element_wait(driver, 10, (By.XPATH, next_button_xpath))
        element.click() 
        page_num += 1

Found 25 rows on Page : 1
Parsing Page : 1
Total rows scraped : 25
Clicking Next Button
Found 25 rows on Page : 2
Parsing Page : 2
Total rows scraped : 50
Clicking Next Button
Found 25 rows on Page : 3
Parsing Page : 3
Total rows scraped : 75
Clicking Next Button
Found 25 rows on Page : 4
Parsing Page : 4
Total rows scraped : 100
Clicking Next Button
Found 25 rows on Page : 5
Parsing Page : 5
Total rows scraped : 125
Done Parsing..


In [16]:
len(table_data)

125

In [17]:
print('Save the data to a CSV')
table_df = pd.DataFrame(table_data)
#print(table_df)
table_df.to_csv('cryptocurrencies.csv', index=None)
#### add timer 

Save the data to a CSV


In [18]:
table_df

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,"43,454.28",-813.63,-1.84%,824.457B,27.905B,27.905B,27.905B,18.973M
1,ETH-USD,Ethereum USD,"2,904.61",-95.07,-3.17%,347.979B,15.28B,15.28B,15.28B,119.802M
2,USDT-USD,Tether USD,1.0003,-0.0001,-0.01%,79.784B,64.919B,64.919B,64.919B,79.757B
3,BNB-USD,Binance Coin USD,406.13,-5.88,-1.43%,67.059B,1.865B,1.865B,1.865B,165.117M
4,USDC-USD,USD Coin USD,0.999648,+0.000331,+0.03%,53.438B,5.028B,5.028B,5.028B,53.457B
...,...,...,...,...,...,...,...,...,...,...
120,BNT-USD,Bancor USD,2.4399,-0.0284,-1.15%,637.247M,37.164M,37.164M,37.164M,261.181M
121,XDC-USD,XDC Network USD,0.051733,-0.001426,-2.68%,636.573M,9.459M,9.459M,9.459M,12.305B
122,TTT1-USD,The Transfer Token USD,6.0655,-0.2034,-3.24%,635.848M,"490,267","490,267","490,267",104.83M
123,RVN-USD,Ravencoin USD,0.058675,-0.003412,-5.50%,614.525M,28.932M,28.932M,28.932M,10.473B


In [ ]:
jovian.commit(project="selenium-yahoo-finance-scraper",git_commit=True)

<IPython.core.display.Javascript object>